# Technical Deliverable

## LightGBM

In [2]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
from  datetime import datetime, timedelta

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Prepare Datasets for Training

## Define the correct data type for each column in the datasets

### *calendar.csv*

In [4]:
# Correct data types for "calendar.csv"
calendarDTypes = {"event_name_1": "category", 
                  "event_name_2": "category", 
                  "event_type_1": "category", 
                  "event_type_2": "category", 
                  "weekday": "category", 
                  'wm_yr_wk': 'int16', 
                  "wday": "int16",
                  "month": "int8", 
                  "year": "int8", 
                  "snap_CA": "int8", 
                  'snap_TX': 'int8', 
                  'snap_WI': 'int8' }

# Read csv file
calendar = pd.read_csv("calendar.csv", 
                       dtype = calendarDTypes)

calendar["date"] = pd.to_datetime(calendar["date"])

# Transform categorical features into integers
for col, colDType in calendarDTypes.items():
    if colDType == "category":
        calendar[col] = calendar[col].cat.codes.astype("int16")
        calendar[col] -= calendar[col].min()

calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,-37,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,-37,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,-37,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,-37,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,-37,d_5,0,0,0,0,1,0,1


In [5]:
reduce_mem_usage(calendar)

Mem. usage decreased to  0.05 Mb (17.1% reduction)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,-37,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,-37,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,-37,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,-37,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,-37,d_5,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,6,5,6,-32,d_1965,0,0,0,0,0,1,1
1965,2016-06-16,11620,4,6,6,-32,d_1966,0,0,0,0,0,0,0
1966,2016-06-17,11620,0,7,6,-32,d_1967,0,0,0,0,0,0,0
1967,2016-06-18,11621,2,1,6,-32,d_1968,0,0,0,0,0,0,0


### *sell_prices.csv*

In [6]:
# Correct data types for "sell_prices.csv"
priceDTypes = {"store_id": "category", 
               "item_id": "category", 
               "wm_yr_wk": "int16",
               "sell_price":"float16"}

# Read csv file
prices = pd.read_csv("sell_prices.csv", 
                     dtype = priceDTypes)

# Transform categorical features into integers
for col, colDType in priceDTypes.items():
    if colDType == "category":
        prices[col] = prices[col].cat.codes.astype("int16")
        prices[col] -= prices[col].min()
        
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,0,0,11325,9.578125
1,0,0,11326,9.578125
2,0,0,11327,8.257812
3,0,0,11328,8.257812
4,0,0,11329,8.257812


In [7]:
reduce_mem_usage(prices)

Mem. usage decreased to 45.67 Mb (12.5% reduction)


,store_id,item_id,wm_yr_wk,sell_price
0,0,0,11325,9.578125
1,0,0,11326,9.578125
2,0,0,11327,8.257812
3,0,0,11328,8.257812
4,0,0,11329,8.257812
...,...,...,...,...
6841116,9,3048,11617,1.000000
6841117,9,3048,11618,1.000000
6841118,9,3048,11619,1.000000
6841119,9,3048,11620,1.000000


### *sales_train_validation.csv*

In [8]:
firstDay = 300
lastDay = 1913

# Use x sales days (columns) for training
numCols = [f"d_{day}" for day in range(firstDay, lastDay+1)]

# Define all categorical columns
catCols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']

# Define the correct data types for "sales_train_validation.csv"
dtype = {numCol: "int16" for numCol in numCols} 
dtype.update({catCol: "category" for catCol in catCols if catCol != "id"})

# Read csv file
ds = pd.read_csv("sales_train_validation.csv", 
                 usecols = catCols + numCols, dtype = dtype)

# Transform categorical features into integers
for col in catCols:
    if col != "id":
        ds[col] = ds[col].cat.codes.astype("int16")
        ds[col] -= ds[col].min()
        
ds = pd.melt(ds,
             id_vars = catCols,
             value_vars = [col for col in ds.columns if col.startswith("d_")],
             var_name = "d",
             value_name = "sales")

# Merge "ds" with "calendar" and "prices" dataframe
ds = ds.merge(calendar, on = "d", copy = False)
ds = ds.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

ds.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_300,0,2011-11-24,11143,...,11,-37,28,2,0,0,0,0,0,3.970703
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_301,0,2011-11-25,11143,...,11,-37,0,0,0,0,0,0,0,3.970703
2,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_300,1,2011-11-24,11143,...,11,-37,28,2,0,0,0,0,0,4.339844
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_301,1,2011-11-25,11143,...,11,-37,0,0,0,0,0,0,0,4.339844
4,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_300,0,2011-11-24,11143,...,11,-37,28,2,0,0,0,0,0,2.480469


In [9]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41551496 entries, 0 to 41551495
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         int16         
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int8          
 11  wday          int8          
 12  month         int8          
 13  year          int8          
 14  event_name_1  int8          
 15  event_type_1  int8          
 16  event_name_2  int8          
 17  event_type_2  int8          
 18  snap_CA       int8          
 19  snap_TX       int8          
 20  snap_WI       int8          
 21  sell_price    float16       
dtypes: datetime64[ns](1), float16(1), int16(7), int8(11), object(2)


In [10]:
reduce_mem_usage(ds)

Mem. usage decreased to 1862.45 Mb (7.8% reduction)


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_300,0,2011-11-24,11143,...,11,-37,28,2,0,0,0,0,0,3.970703
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_301,0,2011-11-25,11143,...,11,-37,0,0,0,0,0,0,0,3.970703
2,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_300,1,2011-11-24,11143,...,11,-37,28,2,0,0,0,0,0,4.339844
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_301,1,2011-11-25,11143,...,11,-37,0,0,0,0,0,0,0,4.339844
4,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_300,0,2011-11-24,11143,...,11,-37,28,2,0,0,0,0,0,2.480469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41551491,FOODS_3_825_WI_3_validation,3046,6,9,2,2,d_1913,0,2016-04-24,11613,...,4,-32,0,0,0,0,0,0,0,3.980469
41551492,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1912,1,2016-04-23,11613,...,4,-32,0,0,0,0,0,0,0,1.280273
41551493,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1913,3,2016-04-24,11613,...,4,-32,0,0,0,0,0,0,0,1.280273
41551494,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1912,0,2016-04-23,11613,...,4,-32,0,0,0,0,0,0,0,1.000000


In [11]:
ds.head().T

,0,1,2,3,4
id,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_005_CA_1_validation
item_id,1,1,3,3,4
dept_id,0,0,0,0,0
store_id,0,0,0,0,0
cat_id,0,0,0,0,0
state_id,0,0,0,0,0
d,d_300,d_301,d_300,d_301,d_300
sales,0,0,1,1,0
date,2011-11-24 00:00:00,2011-11-25 00:00:00,2011-11-24 00:00:00,2011-11-25 00:00:00,2011-11-24 00:00:00
wm_yr_wk,11143,11143,11143,11143,11143


## Create features

### Sales features

In [12]:
dayLags = [7,14,21,28]
lagSalesCols = [f"lag_{dayLag}" for dayLag in dayLags]
for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
    ds[lagSalesCol] = ds[["id","sales"]].groupby("id")["sales"].shift(dayLag)
    
windows = [7,14,21,28]
for window in windows:
    for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
        ds[f"rmean_{dayLag}_{window}"] = ds[["id", lagSalesCol]].groupby("id")[lagSalesCol].transform(lambda x: x.rolling(window).mean())

In [13]:
reduce_mem_usage(ds)

Mem. usage decreased to 3447.51 Mb (58.0% reduction)


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,rmean_21_14,rmean_28_14,rmean_7_21,rmean_14_21,rmean_21_21,rmean_28_21,rmean_7_28,rmean_14_28,rmean_21_28,rmean_28_28
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_300,0,2011-11-24,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_301,0,2011-11-25,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_300,1,2011-11-24,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_301,1,2011-11-25,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_300,0,2011-11-24,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41551491,FOODS_3_825_WI_3_validation,3046,6,9,2,2,d_1913,0,2016-04-24,11613,...,0.571289,0.856934,1.000000,0.904785,0.714355,1.047852,0.928711,0.928711,0.893066,1.250000
41551492,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1912,1,2016-04-23,11613,...,1.071289,1.286133,1.000000,1.095703,1.190430,1.000000,1.036133,1.178711,1.000000,1.107422
41551493,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1913,3,2016-04-24,11613,...,1.071289,1.357422,0.856934,1.142578,1.095703,1.095703,1.036133,1.142578,0.964355,1.250000
41551494,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1912,0,2016-04-23,11613,...,3.000000,2.214844,1.666992,2.427734,2.714844,1.666992,1.821289,2.357422,2.177734,1.786133


### Date features

In [15]:
dateFeatures = {"wday": "weekday",
                "week": "week",
                "month": "month",
                "quarter": "quarter",
                "year": "year",
                "mday": "day"}

for featName, featFunc in dateFeatures.items():
    if featName in ds.columns:
        ds[featName] = ds[featName].astype("int16")
    else:
        if featFunc == "week":
            ds[featName] = getattr(ds["date"].dt.isocalendar(), featFunc).astype("int16")
        else:
            ds[featName] = getattr(ds["date"].dt, featFunc).astype("int16")


In [16]:
reduce_mem_usage(ds)

Mem. usage decreased to 3566.39 Mb (6.2% reduction)


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,rmean_14_21,rmean_21_21,rmean_28_21,rmean_7_28,rmean_14_28,rmean_21_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_300,0,2011-11-24,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,4,24
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_301,0,2011-11-25,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,4,25
2,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_300,1,2011-11-24,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,4,24
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_301,1,2011-11-25,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,4,25
4,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_300,0,2011-11-24,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,4,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41551491,FOODS_3_825_WI_3_validation,3046,6,9,2,2,d_1913,0,2016-04-24,11613,...,0.904785,0.714355,1.047852,0.928711,0.928711,0.893066,1.250000,16,2,24
41551492,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1912,1,2016-04-23,11613,...,1.095703,1.190430,1.000000,1.036133,1.178711,1.000000,1.107422,16,2,23
41551493,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1913,3,2016-04-24,11613,...,1.142578,1.095703,1.095703,1.036133,1.142578,0.964355,1.250000,16,2,24
41551494,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1912,0,2016-04-23,11613,...,2.427734,2.714844,1.666992,1.821289,2.357422,2.177734,1.786133,16,2,23


In [17]:
ds.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,rmean_14_21,rmean_21_21,rmean_28_21,rmean_7_28,rmean_14_28,rmean_21_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_300,0,2011-11-24,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,4,24
1,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_301,0,2011-11-25,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,4,25
2,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_300,1,2011-11-24,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,4,24
3,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_301,1,2011-11-25,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,4,25
4,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_300,0,2011-11-24,11143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,4,24


In [18]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41551496 entries, 0 to 41551495
Data columns (total 45 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int8          
 3   store_id      int8          
 4   cat_id        int8          
 5   state_id      int8          
 6   d             object        
 7   sales         int16         
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int8          
 11  wday          int8          
 12  month         int8          
 13  year          int8          
 14  event_name_1  int8          
 15  event_type_1  int8          
 16  event_name_2  int8          
 17  event_type_2  int8          
 18  snap_CA       int8          
 19  snap_TX       int8          
 20  snap_WI       int8          
 21  sell_price    float16       
 22  lag_7         float16       
 23  lag_14        float16       
 

### Remove unnecessary rows and columns

In [19]:
# Remove all rows with NaN value
ds.dropna(inplace = True)

# Define columns that need to be removed
unusedCols = ["id","date","d", "wm_yr_wk"]
ds = ds.drop(unusedCols,axis=1)

In [20]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39874546 entries, 850137 to 41551495
Data columns (total 41 columns):
 #   Column        Dtype  
---  ------        -----  
 0   item_id       int16  
 1   dept_id       int8   
 2   store_id      int8   
 3   cat_id        int8   
 4   state_id      int8   
 5   sales         int16  
 6   weekday       int8   
 7   wday          int8   
 8   month         int8   
 9   year          int8   
 10  event_name_1  int8   
 11  event_type_1  int8   
 12  event_name_2  int8   
 13  event_type_2  int8   
 14  snap_CA       int8   
 15  snap_TX       int8   
 16  snap_WI       int8   
 17  sell_price    float16
 18  lag_7         float16
 19  lag_14        float16
 20  lag_21        float16
 21  lag_28        float16
 22  rmean_7_7     float16
 23  rmean_14_7    float16
 24  rmean_21_7    float16
 25  rmean_28_7    float16
 26  rmean_7_14    float16
 27  rmean_14_14   float16
 28  rmean_21_14   float16
 29  rmean_28_14   float16
 30  rmean_7_21    fl

In [21]:
y_train = ds["sales"]

In [22]:
X_train = ds.loc[ : , ds.columns != 'sales']

In [23]:
trainCols = X_train.columns 

In [24]:
trainCols

Index(['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'weekday',
       'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7',
       'lag_14', 'lag_21', 'lag_28', 'rmean_7_7', 'rmean_14_7', 'rmean_21_7',
       'rmean_28_7', 'rmean_7_14', 'rmean_14_14', 'rmean_21_14', 'rmean_28_14',
       'rmean_7_21', 'rmean_14_21', 'rmean_21_21', 'rmean_28_21', 'rmean_7_28',
       'rmean_14_28', 'rmean_21_28', 'rmean_28_28', 'week', 'quarter', 'mday'],
      dtype='object')

In [25]:
del ds

### Split dataset into train and validation set

In [26]:
np.random.seed(777)

# Define categorical features
catFeats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + \
           ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]

validInds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
trainInds = np.setdiff1d(X_train.index.values, validInds)

trainData = lgb.Dataset(X_train.loc[trainInds], label = y_train.loc[trainInds], 
                        categorical_feature = catFeats, free_raw_data = False)
validData = lgb.Dataset(X_train.loc[validInds], label = y_train.loc[validInds],
                        categorical_feature = catFeats, free_raw_data = False)

In [27]:
del X_train, y_train, validInds, trainInds ; gc.collect()

479

# Model

In [32]:
params = {
          "objective" : "poisson",
          "metric" :"rmse",
          "force_row_wise" : True,
          "learning_rate" : 0.075,
          "sub_row" : 0.75,
          "bagging_freq" : 1,
          "lambda_l2" : 0.1,
          "metric": ["rmse"],
          'verbosity': 20,
          'num_iterations' : 1200,
          'num_leaves': 128,
          "min_data_in_leaf": 100,
         }

In [33]:
# Train LightGBM model
m_lgb = lgb.train(params, trainData, valid_sets = [validData])

/Users/amanrajvardhan/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.310966
[LightGBM] [Info] Total Bins 7857
[LightGBM] [Info] Number of data points in the train set: 37874546, number of used features: 40
[LightGBM] [Info] Start training from score 0.308587
[LightGBM] [Debug] Re-bagging, using 28408329 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 10
[LightGBM] [Debug] Re-bagging, using 28406030 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 12
[LightGBM] [Debug] Re-bagging, using 28405050 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 12
[LightGBM] [Debug] Re-bagging, using 28405816 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 14
[

[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 13
[LightGBM] [Debug] Re-bagging, using 28405706 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 14
[LightGBM] [Debug] Re-bagging, using 28405676 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 13
[LightGBM] [Debug] Re-bagging, using 28404634 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 15
[LightGBM] [Debug] Re-bagging, using 28414102 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 15
[LightGBM] [Debug] Re-bagging, using 28406244 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 14
[LightGBM] [Debug] Re-bagging, using 28399415 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 20
[LightGBM] [Debug] Re-bagging, using 28407107 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 15
[LightGBM] [Debug] Re-bagging, using 2840629

[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 18
[LightGBM] [Debug] Re-bagging, using 28406476 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 17
[LightGBM] [Debug] Re-bagging, using 28404600 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 34
[LightGBM] [Debug] Re-bagging, using 28406340 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 18
[LightGBM] [Debug] Re-bagging, using 28406456 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 23
[LightGBM] [Debug] Re-bagging, using 28405534 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 34
[LightGBM] [Debug] Re-bagging, using 28405598 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 16
[LightGBM] [Debug] Re-bagging, using 28405017 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 29
[LightGBM] [Debug] Re-bagging, using 2840625

[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 26
[LightGBM] [Debug] Re-bagging, using 28405110 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 29
[LightGBM] [Debug] Re-bagging, using 28404786 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 18
[LightGBM] [Debug] Re-bagging, using 28405790 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 40
[LightGBM] [Debug] Re-bagging, using 28403025 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 32
[LightGBM] [Debug] Re-bagging, using 28405973 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 28
[LightGBM] [Debug] Re-bagging, using 28406242 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 28
[LightGBM] [Debug] Re-bagging, using 28406024 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 29
[LightGBM] [Debug] Re-bagging, using 2840672

...

[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 46
[LightGBM] [Debug] Re-bagging, using 28406623 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 30
[LightGBM] [Debug] Re-bagging, using 28403171 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 26
[LightGBM] [Debug] Re-bagging, using 28406446 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 35
[LightGBM] [Debug] Re-bagging, using 28407381 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 22
[LightGBM] [Debug] Re-bagging, using 28407610 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 35
[LightGBM] [Debug] Re-bagging, using 28405887 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 24
[LightGBM] [Debug] Re-bagging, using 28407156 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 37
[LightGBM] [Debug] Re-bagging, using 2839734

[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 35
[LightGBM] [Debug] Re-bagging, using 28407271 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 37
[LightGBM] [Debug] Re-bagging, using 28406559 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 42
[LightGBM] [Debug] Re-bagging, using 28406326 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 46
[LightGBM] [Debug] Re-bagging, using 28415254 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 43
[LightGBM] [Debug] Re-bagging, using 28406229 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 41
[LightGBM] [Debug] Re-bagging, using 28406341 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 48
[LightGBM] [Debug] Re-bagging, using 28405633 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 51
[LightGBM] [Debug] Re-bagging, using 2840685

[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 55
[LightGBM] [Debug] Re-bagging, using 28406359 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 52
[LightGBM] [Debug] Re-bagging, using 28406558 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 25
[LightGBM] [Debug] Re-bagging, using 28408860 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 35
[LightGBM] [Debug] Re-bagging, using 28405914 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 36
[LightGBM] [Debug] Re-bagging, using 28405570 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 45
[LightGBM] [Debug] Re-bagging, using 28404683 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 20
[LightGBM] [Debug] Re-bagging, using 28400405 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 15
[LightGBM] [Debug] Re-bagging, using 2840526

[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 40
[LightGBM] [Debug] Re-bagging, using 28406810 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 54
[LightGBM] [Debug] Re-bagging, using 28406445 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 35
[LightGBM] [Debug] Re-bagging, using 28403918 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 48
[LightGBM] [Debug] Re-bagging, using 28403727 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 35
[LightGBM] [Debug] Re-bagging, using 28407597 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 39
[LightGBM] [Debug] Re-bagging, using 28402202 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 28
[LightGBM] [Debug] Re-bagging, using 28405808 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 36
[LightGBM] [Debug] Re-bagging, using 2840648

[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 37
[LightGBM] [Debug] Re-bagging, using 28407392 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 37
[LightGBM] [Debug] Re-bagging, using 28405937 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 21
[LightGBM] [Debug] Re-bagging, using 28405714 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 44
[LightGBM] [Debug] Re-bagging, using 28404836 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 28
[LightGBM] [Debug] Re-bagging, using 28403294 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 34
[LightGBM] [Debug] Re-bagging, using 28406183 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 38
[LightGBM] [Debug] Re-bagging, using 28405198 data to train
[LightGBM] [Debug] Trained a tree with leaves = 128 and depth = 33
[LightGBM] [Debug] Re-bagging, using 2840602

In [34]:
import pickle

In [35]:
# Save the model
m_lgb.save_model("model.lgb")

In [36]:
file = 'trained_model.pkl'

In [37]:
pickle.dump(m_lgb, open(file, 'wb'))

In [38]:
loaded_model = pickle.load(open(file, 'rb'))

In [39]:
loaded_model.eval_valid(feval=())

[]

# Predictions

In [42]:
# Last day used for training
trLast = 1913
# Maximum lag day
maxLags = 60

# Create dataset for predictions
def create_ds():
    
    startDay = trLast - maxLags
    
    numCols = [f"d_{day}" for day in range(startDay, trLast + 1)]
    catCols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    
    dtype = {numCol:"float32" for numCol in numCols} 
    dtype.update({catCol: "category" for catCol in catCols if catCol != "id"})
    
    ds = pd.read_csv("sales_train_validation.csv", 
                     usecols = catCols + numCols, dtype = dtype)
    reduce_mem_usage(ds)
    for col in catCols:
        if col != "id":
            ds[col] = ds[col].cat.codes.astype("int16")
            ds[col] -= ds[col].min()
    
    for day in range(trLast + 1, trLast+ 28 +1):
        ds[f"d_{day}"] = np.nan
    
    ds = pd.melt(ds,
                 id_vars = catCols,
                 value_vars = [col for col in ds.columns if col.startswith("d_")],
                 var_name = "d",
                 value_name = "sales")
    
    ds = ds.merge(calendar, on = "d", copy = False)
    ds = ds.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return ds

def create_features(ds):          
    dayLags = [7,14,21,28]
    lagSalesCols = [f"lag_{dayLag}" for dayLag in dayLags]
    for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
        ds[lagSalesCol] = ds[["id","sales"]].groupby("id")["sales"].shift(dayLag)

    windows = [7,14,21,28]
    for window in windows:
        for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
            ds[f"rmean_{dayLag}_{window}"] = ds[["id", lagSalesCol]].groupby("id")[lagSalesCol].transform(lambda x: x.rolling(window).mean())
          
    dateFeatures = {"wday": "weekday",
                "week": "week",
                "month": "month",
                "quarter": "quarter",
                "year": "year",
                "mday": "day"}

    for featName, featFunc in dateFeatures.items():
        if featName in ds.columns:
            ds[featName] = ds[featName].astype("int16")
        else:
            if featFunc == "week":
                ds[featName] = getattr(ds["date"].dt.isocalendar(), featFunc).astype("int16")
            else:
                ds[featName] = getattr(ds["date"].dt, featFunc).astype("int16")

In [43]:
fday = datetime(2016,4, 25) 
alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)] * len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_ds()
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = reduce_mem_usage(te[(te['date'] >= day - timedelta(days=maxLags)) & (te['date'] <= day)]).copy()
        create_features(tst)
        reduce_mem_usage(tst)
        tst = tst.loc[tst['date'] == day , trainCols]
        te.loc[te['date'] == day, "sales"] = alpha * loaded_model.predict(tst) # magic multiplier by kyakovlev

    te_sub = te.loc[te['date'] >= fday, ["id", "sales"]].copy()
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = reduce_mem_usage(sub).copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submissionlgb.csv",index=False)

Mem. usage decreased to  4.04 Mb (46.7% reduction)
0 2016-04-25 00:00:00
Mem. usage decreased to 97.56 Mb (15.4% reduction)


/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int16)
/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int8)
/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Mem. usage decreased to 173.83 Mb (51.0% reduction)
1 2016-04-26 00:00:00
Mem. usage decreased to 97.56 Mb (15.4% reduction)


/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int16)
/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int8)
/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Mem. usage decreased to 173.83 Mb (51.0% reduction)
2 2016-04-27 00:00:00
Mem. usage decreased to 97.56 Mb (15.4% reduction)


...

Mem. usage decreased to 97.56 Mb (15.4% reduction)
Mem. usage decreased to 173.83 Mb (51.0% reduction)
26 2016-05-21 00:00:00


/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int16)
/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int8)
/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Mem. usage decreased to 97.56 Mb (15.4% reduction)
Mem. usage decreased to 173.83 Mb (51.0% reduction)
27 2016-05-22 00:00:00
Mem. usage decreased to 97.56 Mb (15.4% reduction)


/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int16)
/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int8)
/var/folders/s7/j1_y6ym57n59dqv2fsslyt5h0000gn/T/ipykernel_8881/3652629765.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Mem. usage decreased to 173.83 Mb (51.0% reduction)
2 1.018 0.3333333333333333
Mem. usage decreased to  1.86 Mb (72.4% reduction)
